In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
import os

warnings.simplefilter('ignore')
plt.rcParams['figure.figsize'] = (15,12)
plt.style.use('seaborn')

In [2]:
df = pd.read_csv('training_tweets.csv').drop(columns='Unnamed: 0')

In [3]:
df.head()

,content,sentiment
0,Happy Mama's day to all mothers,love
1,@LysdelTellez I am lost. Please help me find a...,worry
2,"@BoomKatt yes yes I AM, networking whore to th...",happiness
3,you@snapplynn Wish that would have been your t...,neutral
4,now i am doing the MicroEconomics project iha...,worry


In [4]:
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/feedingthemachine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/feedingthemachine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
df['content'] = df.content.apply(lambda x: lemmatizer.lemmatize(x))

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
# instanciamos un objeto
count_vectorizer=CountVectorizer(stop_words='english', max_features=5000)
# Implementamos los pasos fit y transform
count_vectorizer_fit = count_vectorizer.fit_transform(df.content)
# Extraemos tokens (palabras)
words = count_vectorizer.get_feature_names()
# extraemos frecuencia
words_freq = count_vectorizer_fit.toarray().sum(axis=0)

df_words = pd.DataFrame({'Word':words, 'Frec':words_freq})

top5000 = df_words.sort_values(by='Frec', ascending=False)

In [8]:
top5000.head()

,Word,Frec
1176,day,2403
2363,just,2369
1867,good,1736
2561,like,1444
3519,quot,1399


In [9]:
isineutral = np.isin(df['sentiment'], ['neutral'])
isin = np.isin(df['sentiment'], ['happiness','love','surprise','fun','relief','enthusiasm','positive'])

In [10]:
df['sentiment'] = np.where(isineutral, np.random.choice(['positive', 'negative']),np.where(isin,'positive','negative'))

In [11]:
df

,content,sentiment
0,Happy Mama's day to all mothers,positive
1,@LysdelTellez I am lost. Please help me find a...,negative
2,"@BoomKatt yes yes I AM, networking whore to th...",positive
3,you@snapplynn Wish that would have been your t...,negative
4,now i am doing the MicroEconomics project iha...,negative
5,@XThe_Happy_EmoX lmao. Lucky! It's 10 minutes ...,negative
6,@jackEO313 at first i thought bar life meant y...,positive
7,@kyleandjackieo Fighter - Christina Aguilera. ...,positive
8,@taxidermi I was watching Parental Control,negative
9,"me 10 - 630, andrew 8 - 4. No idea about the r...",positive
